***
***
# 使用Turicreate进行音乐推荐
***
***


In [2]:
import turicreate as tc

# 下载数据
http://s3.amazonaws.com/dato-datasets/millionsong/10000.txt


In [3]:
#train_file = 'http://s3.amazonaws.com/dato-datasets/millionsong/10000.txt'
train_file = '/Users/datalab/bigdata/cjc/millionsong/song_usage_10000.txt'
sf = tc.SFrame.read_csv(train_file, header=False, delimiter='\t', verbose=False)
sf = sf.rename({'X1':'user_id', 'X2':'music_id', 'X3':'rating'})

In [4]:
train_set, test_set = sf.random_split(0.8, seed=1)

In [5]:
popularity_model = tc.popularity_recommender.create(train_set, 
                                                    'user_id', 'music_id', 
                                                    target = 'rating')

Preparing data set.

Data has 1599753 observations with 76085 users and 10000 items.

Data prepared in: 0.907738s

1599753 observations to process; with 10000 unique items.

In [7]:
item_sim_model = tc.item_similarity_recommender.create(train_set, 
                                                       'user_id', 'music_id', 
                                                       target = 'rating', 
                                                       similarity_type='cosine')

Preparing data set.

Data has 1599753 observations with 76085 users and 10000 items.

Data prepared in: 0.939059s

Training model from provided data.

Gathering per-item and per-user statistics.

...

| 957.28ms                            | 100              | 10000           |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 2.00431s

In [8]:
factorization_machine_model = tc.recommender.factorization_recommender.create(train_set, 
                                                                              'user_id', 'music_id',
                                                                              target='rating')

Preparing data set.

Data has 1599753 observations with 76085 users and 10000 items.

Data prepared in: 0.930001s

Training factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 8.16243

Final training RMSE: 2.85534

In [9]:
len(train_set)

1599753

In [10]:
result = tc.recommender.util.compare_models(test_set, 
                                            [popularity_model, item_sim_model, factorization_machine_model],
                                            user_sample=.5, skip_set=train_set)

compare_models: using 34354 users to estimate model performance
PROGRESS: Evaluate model M0


recommendations finished on 1000/34354 queries. users per second: 19077.4

recommendations finished on 2000/34354 queries. users per second: 21071.3

recommendations finished on 3000/34354 queries. users per second: 21793

recommendations finished on 4000/34354 queries. users per second: 22081.5

...

recommendations finished on 31000/34354 queries. users per second: 16228.5

recommendations finished on 32000/34354 queries. users per second: 16176.8

recommendations finished on 33000/34354 queries. users per second: 16184.1

recommendations finished on 34000/34354 queries. users per second: 16170.4


Precision and recall summary statistics by cutoff
+--------+------------------------+------------------------+
| cutoff |      mean_recall       |     mean_precision     |
+--------+------------------------+------------------------+
|   1    |  7.0023278723497e-05   | 0.0004075216859754317  |
|   2    | 0.00015701647350617692 | 0.0004220760319031288  |
|   3    | 0.00025150526145133087 | 0.00041722458326056295 |
|   4    | 0.00034565751961335053 | 0.00043663037783081874 |
|   5    | 0.00046127750964837765 | 0.0004424521162018989  |
|   6    | 0.0006061722786786824  | 0.0004899963128990292  |
|   7    |  0.00070561257944388   | 0.0005197980688462193  |
|   8    | 0.0008535975421092137  | 0.0005457879722885229  |
|   9    | 0.0009773205690967743  | 0.0005433622479672466  |
|   10   | 0.0011201566127395915  | 0.0005647086219945271  |
+--------+------------------------+------------------------+
[10 rows x 3 columns]


Overall RMSE: 7.780049286375036

Per User RMSE (best)
+----------------

In [12]:
K = 10
users = gl.SArray(sf['user_id'].unique().head(100))

In [13]:
recs = item_sim_model.recommend(users=users, k=K)
recs.head()

user_id,music_id,score,rank
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOXUQNR12AF72A69D6,3.022422651449839,1
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOUFAZA12AC3DFAB20,1.3368427753448486,2
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOSFSTC12A8C141219,1.091982126235962,3
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOVIWFP12A58A7D1BD,1.045163869857788,4
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOBMTQD12AB01833D0,1.0294516881306965,5
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOCMNRG12AB0189D3F,0.9756437937418619,6
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOXOHUM12A67ADC826,0.9506873289744059,7
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOWBFVW12A6D4F612B,0.9092370669047037,8
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOXFYTY127E9433E7D,0.8977278073628744,9
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,SOYBLYP12A58A79D32,0.8970928192138672,10
